# Automation of Microfluidic Experiments -- 
## 4 Pump Control

Welcome to this short explantion of the various features of the 4pump_control.py software
This code was written to control a 4 input microfluidic mixer. 

The main idea behind the project was to make a 4 input microfluidic chip, that would be able to take 2 chemicals, and dilute each one independently, and search for various interesting chemical features at different concentrations and stoichometries of chemicals. 

This code is based on the 2pump_control.py file. If you wish to understand the code itself, it would be best to start with understanding the 2pump_control.py code, and then moving on to the 4pump_control.py code. If you wish to simply understand the features of the code, this explainer should be sufficient, without having to go through the 2pump_control explainer.

The aim of the code was to control a 4 input microfluidic mixer chip, that could mix 2 substances in various proportions and various dilutions of each susbtance. Due to time limitation, only the dilution target was implemented, so the code, combined with the mixrofluidic chip, could dilute 2 subtances individually, and therefore vary the ratio of one substances to another substance at the measurement point. This was done by varying the flow rate ratio (FRR) of the dilution agent and substances. However, the code is not written to vary the FRR of the two already diluted streams.

There are no large obstacles in the development of that code. Currently, the program is configured to have a 1:1 FRR of the two diluted subtance streams.

But to understand the program more deeply, let us start at the beginning. Let us load the appropriate file. Note that this file relies the connect.py file, so both need to be present in the same directory for the program to function as intended.

In [27]:
%run 4pump_control.py

Initially, we have to create a dictionary of available pumps. We will make this dictionary with the pump_tester() function. It can take baudrate as a optional parameter.

It iterates through all detected COM ports, assumes a pump is connected and cycles the pump on and off. It then asks the user if a pump is connected. If yes, the user is meant to press enter, if not, any other input is acceptable. 

Note that this process stalls if Bluetooth is turned on, as the bluetooth connection is also seen as COM ports, but they do not behave the same way as other COM ports.

If a pump is connected, and enter is pressed, the function then asks for a pump number. The pumps should be numbered "1", "2", "3", "4", with pumps "1" and "3" containing the substnaces of interest, and pumps "2" and "4" containing the dilution agents.

### Pump compatibility
This project used Chemyx Inc. Fusion 100 & 300 syringe pumps, which connect via serial over USB. Chemyx Inc. provides a basic interface which is loaded by the 4pump_control.py file (the connect.py file). This interface is the file connect.py.

If a different set of pumps were to be used, if the producer produced a different python control program, the connect.py file could be feasibly simply replaced, with a few adjustments to the Pump class. 

Otherwise, if the control has to be implemented directly, it can be implemented into the Pump class. Once the Pump class is adjusted, the rest of the program should function without any other edits.

In [2]:
# does not work with bluetooth turned on... gets stuck on bluetooth COM ports
pump_dictionary = pump_tester()

available COM ports are: []


There are also dotstrings for all the functions and methods within the 4pump_control.py program. So if anything is unfamilar, you can type in:


    help(__function__)


    help(pump_tester)

In [10]:
help(pump_tester)

Help on function pump_tester in module __main__:

pump_tester(baudrate=38400)
    Summary:
        uses connect.py from pump manufacturer website to check for open serial ports
        and then generate and return a list of pumps and ports
        
        need to define pumps with numbers 1 - 4, pump 1 and pump 3 have chemicals, 
        pump 2 and 4 contain dilution agents
        
        DOES NOT WORK ON WINDOWS IF BLUETOOTH TURNED ON
        (gets stuck on bluetooth COM ports)
    Optional Arguments:
        baudrate (int): baudrate used for communication with Chemyx Fusion Pumps



Once the pump dictionary is created, we can move on to the main section of the program.
# The Complex Experiment class

In [9]:
help(complex_experiment)

Help on class complex_experiment in module __main__:

class complex_experiment(builtins.object)
 |  complex_experiment(pump_dic, pump1_max_vol, pump2_max_vol, pump3_max_vol, pump4_max_vol, overall_flow, equilibration_time, target_value, path123, units, syringe_diameter, syringe_volume, _pumps='ON', communication_rate=38400, colour='R', load_dic_path=0)
 |  
 |  DESCRIPTION
 |      Main working class of the program... is responsible for file structure generation, loading a dictionary
 |      of available pumps and initializing them internally as Pump objects, based on the class above
 |      also requires the fill volume of syringes to auto stop any experiments before fluid is exhausted
 |      was imagined to be able to run multiple experiments, however, due to how the files are saved
 |      it makes more sense to "end" the class (closes the open serial connections to the pumps)
 |      and create a new iteration of the complex_experiment class for new experiments or new trials
 |  
 

The complex_experiment class contains all the functions of the program. You can see the input parameters below. Most should be fairly intuitive. The pump max volume refers to the volume of fluid inside the pumps, with the syringe_volume refering to the actual maximum volume the syringe pump could contain. 
The overall_flow is the overall flow rate of the four pumps combined. Each two pump pairs (1&2 and 3&4) output half the overall flow.


In [18]:
pump_dictionary # defined earlier
pump1_max_volume = 10 #in mL
pump2_max_volume = 10
pump3_max_volume = 10
pump4_max_volume = 10

overall_flow = 0.8  #in units set below
equilibration_time = 45  #in seconds
target_sensor_readout = 100  #explained later
path123 = os.path.dirname(os.path.abspath("2pump_control.py")) #any path could be used as input
units = "mL/min" #other option is μL/min
syringe_diameter = 15.68 #in mm
syringe_volume = 12  #in mL
_pumps = "ON"  #should be "OFF" if pumps are not connected, but many features will not work

# Optional Arguments
communication_rate = 38400  # baudrate to be used
colour = "R"  # colour channel to be used by data loading and automations methods
load_dic_path = 0  # explained later

exp2 = complex_experiment(pump_dictionary, pump1_max_volume, pump2_max_volume, pump3_max_vol, pump4_max_vol, 
                          overall_flow, equilibration_time, target_sensor_readout, path123, units, 
                          syringe_diameter, syringe_volume, _pumps,
                          communication_rate, colour, load_dic_path)

Once the class is initialized, it will create a series of folders in the path of path123. It also makes a .pkl file, whose use will be demonstrated later.

## Methods of the Experiment class
All the different features of the 4pump_control software are implemented as methods of the complex_experiment class, with the exception of the pump3only_auto_mapping function explained at the end. 

You can see the methods in the complex_experiment class in the following way:

In [12]:
method_list = [method for method in dir(complex_experiment) if method.startswith('__') is False]
print(method_list)

['aim_ximea', 'append_vol', 'check_volume_to_pump', 'end', 'get_reading', 'linear_surface_fit', 'linear_surface_plane_function', 'load_the_map', 'load_the_map_for_scatter', 'load_the_map_only_1_pump_chaning_FRR', 'map_the_space', 'map_the_space_logspacing', 'map_the_space_pump1_list', 'minimize_fit_func', 'plot_map', 'pumping', 'quadratic_surface_fit', 'quadratic_surface_plane_funtion', 'reset_volume', 'runall', 'set_diameter', 'set_units', 'steady_state_time_finder', 'stopall', 'take_reading', 'totvolume_pumped', 'update_vol', 'volume_pumped']


You can see, there are a number of methods, but not all of them are equally useful...

Let us start with the assumptions the program makes about the experimental setup. It is assumed that measurements will be taken using a Ximea Camera, through the Ximea CamTool GUI program. Ximea CamTool can be found on the Ximea website, as well as a version of it is included in this github repo. 

https://www.ximea.com/support/wiki/allprod/XIMEA_CamTool

The 4pump_control.py software records data by taking over control of the mouse and keyboard. It saves images using the CamTool, but those are meant only for reference. All the data processing is done by also taking a line profile, and using its average as an intensity reading. The way it is set up, it assumes it is working on a 4K screen on Windows, with Ximea CamTool maximized on the screen. 

-- If you wish to use a different resolution screen, you will need to edit the positions that the pyautogui module moves the mouse to to click to save the line profile in the Ximea CamTool GUI. A version where image detection was used to guide where the mouse clicks, so that it could be used on any resolution display was made, however it would sometimes fail, and was not reliable enough.

![CAMTOOL](XimeaCamTool.png)

The line profile in the lower section of the screen has to be there. It can be enabled in CamTool with:

Tools -> Line Profile. 

The line profile has to be lined up over the region of interest. In the project, a line width averaging of 251 pixels was used. The program assumes it is working with a black and white camera. If a colour camera is used, the colour channel to be used has to be defined in the complex_experiment class. See the dotstring for more details.

## Operating the 4pump_control program


The first thing to do intially, before any measurements are done: the steady state equilibration time for the set up neets to be found. That is, the time it takes from the pumps chaning their relative pumping speeds to that change being visible in the measurement area of the setup, including the time it takes for that changed flow to stablize. 

For this, the steady_state_time_finder method was written, that starts with pump 1 & 3 pumping at half power, i.e. with half the flow rate of the overall flow rate, and then once enter is pressed, the pump power increases to 3/4 power. Then, enter has to be pressed again once the flow has stabilized.

In [ ]:
print(help(exp.steady_state_time_finder()))
exp2.steady_state_time_finder()

Once you know this steady state time for a particular set up, you can just define it in the intialization of the class. You can also use the optional arguments in the steady_state_time_finder() to vary the initial pump power of pump 1 & 3, and by how much it increases after pressing enter.

## A note on pump power

In this work, pump power is refered to many times. It is best to explain it while thinking of two pumps with a constant overall output. Pump power, of each pump making up the pair, is a parameter, that goes from 0 to 1, with 0 corresponding to a pump stopped, and 1, corresponding to the pump pumping at the overall output. Therefore, for each pair of pumps, the sum of their pump powers must be equal to 1. In this code, the pairs of pumps and pump 1 & 2 and pump 3 & 4. Together, these pairs pump out the overall flow defined in the class initialization, so each pair's overall output is half on the overall flow defined.

Note that due to how the pumps handle an input rate of 0, and because it was not necessary for the project, the code currently does not function correctly when 0 or 1 is used as an input for pump power. Therefore, if manually defining pump power, it should be >0 and <1.

# Cycling through different concentrations
As briefly mentioned at the beginning, the program was written to investigate different concentrations of a given substance. The main way that this is done is through the use of "maps". A map is simply a series of sensor measurements (via the Ximea CamTool line profile) corresponding to different flow rate ratios (FRRs) of the pump pairs. To generate a map, first you have to aim Ximea CamTool to the correct save location:

    exp2.aim_ximea(exp.path_fold_map))

Once you run this cell, you have 2 seconds to alt tab to the maximized Ximea CamTool window. The exp.path_fold_map attribute is simply the path to the folder the class initialization made to store the save files from generating the map. The other 2 folder locations will be explained later.

You need to run the aim_ximea(exp2.path_fold_map) every time before running a map. 

In [ ]:
exp2.aim_ximea(exp2.path_fold_map)

Once Ximea is "aimed", and the syringe pumps are loaded with filled syringes etc., the concentration space can be mapped. Again, after running the following cell, you have to switch to the maximized Ximea CamTool window for the duration of the experiment.

In [ ]:
bounds1 = (0.01, 0.99) ## corresponds to the maximum and minimum pump 1 power that will be measured

bounds2 = (0.01, 0.99) ## corresponds to the maximum and minimum pump 3 power that will be measured

n1_samples = 10 ## number of sample points to be take in the map in the pump 1 & 2 space

n2_samples = 10 ## number of sample points to be take in the map in the pump 3 & 4 space

reverse1_flag = True ## if true, will start with high pump 1 power, if False, will start with low pump 1 power

reverse2_flag = True ## if true, will start with high pump 3 power, if False, will start with low pump 3 power
        
exp2.map_the_space(self, bounds1, bounds2, n1_samples, n2_samples, reverse1_flag, reverse2_flag)

The pump 1 & 3 powers and various other parameters are saved in a .txt file called map *time*. It has a header labelling the data columns. 

## Note on pump performance
With the Chemyx Inc Fusion pumps, it was found that at large FRRs, (e.g. pump power > 0.9 or <0.1), the high pumping pump can overwhelm the other pump, and cause no output of one of the pumps. This does not occur when the pump power is reduced gradually. The reverse flag was used to do this semi manually, for example by setting the bounds as (0.1, 0.001) and then having the reverse_flag as True. This kind of splitting up the maps is done automatically in the later mapping functions.

In the map above, the spacing between the sample points is linear in the "pump power" space, i.e. the is a linear spacing of tested concentrations, that correspond to the (pump 1 power)*(syringe concentration).

If you would want to keep a constant dilution of one substance, and only vary the second substance, you could use the following method to manually define the list of pump 1 powers to be tested and only input 1 pump power.
    
    exp2.map_the_space_pump1_list(pump1_power_array, bounds2, n2_samples, reverse1_flag, reverse2_flag)

In [ ]:
exp2.aim_ximea(exp2.path_fold_map)

pump1_power_array = [0.25] ## the pump 1 powers that will be tested

bounds2 = (0.01, 0.99) ## corresponds to the maximum and minimum pump 3 power that will be measured

n2_samples = 10 ## number of sample points to be take in the map in the pump 3 & 4 space

reverse1_flag = True ## if true, will reverse the pump1_power_array

reverse2_flag = True ## if true, will start with high pump 3 power, if False, will start with low pump 3 power

exp2.map_the_space_pump1_list(pump1_power_array, bounds2, n2_samples, reverse1_flag, reverse2_flag)

### Logarithmic Spacing of mapped Points

If you wish to change the spacing in the above method, so that the pump 3 sample points are equally spaced on a logarithmic plot, you can use the following method.

Note that for logspacing method:

The pump 3 powers are analyzed, and if there are points above pump 3 power = 0.5 and points below, first, the points above pump 3 power = 0.5 are performed, starting with the point closest to 0.5, and then the same is done for the points below 0.5 pump 3 power, again starting with the point closest to 0.5.

This is done to improve pump performance. As mentioned earlier, pumps perform better when they go towards very large (or very small) FRRs gradually, and when they start at a more balanced FRR.

In [ ]:
exp2.aim_ximea(exp2.path_fold_map)

pump1_power_array = [0.25] ## the pump 1 powers that will be tested

bounds2 = (-3, -0.01) ## corresponds to the maximum and minimum pump 3 power that will be measured
                      ## Note that the bounds are now as powers of ten, i.e. -3 corresponds to 0.001, 
                      ## and 0 would correspond to 1 as pump 3 power

n2_samples = 10 ## number of sample points to be take in the map in the pump 3 & 4 space

map_the_space_logspacing(pump1_power_array, bounds2, n2_samples):

# Data Analysis

After a map is created, you can use the same class to load the map:

                     #optional argument
                     #by default it is "R"
       colour = "R"  #colour channel to be used... see note below

                     
       ratio1_data, ratio2_data, intensity_array = exp2.load_the_map(colour)
       
The data returned is formated to be plotted with as a 3D surface with matplotlib. It returns flow rate ratios of the pump pairs as the x and y data, even though it is probably more useful to analyze data using pump power, as this corresponds to concentration.

In [ ]:
colour = "R"
ratio1_data, ratio2_data, intensity_array = exp2.load_the_map(colour)

You can turn the FRR data into pump power simply by doing the following.

In [ ]:
pump1_power = (ratio1_data/(1+ratio1_data))
pump3_power = (ratio3_data/(1+ratio3_data))

Note: The colour channel to be used depends on two things. If the camera is black and white, because of how the line profile txt file is formatted, you have to set the colour to the "R", red, channel. If it is a colour camera, you can set it to "R", "G", "B", "BW", "RIN", "BIN", "GIN", "BWIN" for red, green, blue, black and white, red inverse, blue inverse, green inverse and black and white inverse respectively. The inverse colours are simply "255 - sensor readout". The BW mode for a colour camera combines the RGB values and averages them and gives that as an intensity value. 

You can also load the data in a format to make a 3D scatter plot
    
    exp2.load_the_map_for_scatter() 

In [ ]:
_1D_pump1_power, _1D_pump3_power, _1D_ratio1_data, _1D_ratio2_data, _1D_intensity_array = exp2.load_the_map_for_scatter() 
# this method uses the colour channel defined in the initialization
# you can change it with:
# exp2.colour = "G"

Or if you have just 1 pump 1 power setting, and want to have the data for a 2D plot:

In [ ]:
pump1_power, pump3_power, ratio1_data, ratio2_data, intensity, int_std = exp2.load_the_map_only_1_pump_chaning_FRR()
#again colour is defined by the internal value

You can also quickly plot the map, with minimal data analysis but also with great ease of use, if you want to quickly look at the data after collection. It is a 3D scatter plot of the data collected.

In [ ]:
colour = "R" #this is the default setting of the optional argument "colour"
exp2.plot_map(colour)

### Fitting Models

If you wish to perform a linear or quadratic surface fit to the data collected (assuming you varied both pump pairs), you can do so with the following functions:

    exp2.linear_surface_fit(X, Y, Z)
        returns a1, a2, c
    exp2.quadratic_surface_fit(X, Y, Z)
        returns a1, a2, a3, a4, a5, c
        
These are quite standard functions, they were implemented into the program, but never actually used on collected data.

# Target Finding function -- Advanced Implementation
As in the 2pump_control.py program, there is a quadratic fitting function, that searches for peaks and a set target sensor readout. It is based on the minimize_scalar function, from scipy.optimize, and has been implemented to iterate over various sensor readouts, creating a feedback loop that then guides the program towards the target value, or a peak. It does not rely on any previously collected data. 

It first asks the user for a pump 1 power input, to set a pump 1 power that will be constant during the searching, and then it will vary the pump 3 power to achieve either the target value as defined in the initialization, or a peak:

In [ ]:
## To Find the target value as a sensor readout:

mybounds = (0.001, 0.999) # bounds of possible pump 1 power in the search of the target value
x_atol = (0.02) # absolute tolerance of pump power in search of target value
maxiter = 7 # maximum number of iterations allowed in search


exp.target_finding_function(mybounds, x_atol, maxiter)

The above parameters are the default values of those parameters, i.e. if you wanted to run those parameters, you could just enter the code:

    exp.target_finding_function()
The bounds are limitted by the definition of pump power, i.e. >0 and <1, but very small numbers such as 0.0000001 with overall flow of e.g. 8 mL / min, are also misinterpreted by the Chemyx Inc. Fusion pumps. 

The data is saved in the auto_fit folder. At the end, a .txt file is created with the pump 1 power and the sensor readouts. It has the name:

    target fitting dataset itr *iteration number*.txt
### Changing internal attributes
If you wish to change the taget value attribute in an existing instances of the experiment class, you can do this with:

    exp.target_value = 25
You can do this with any of the attributes defined in the class. So you could also do it for e.g.
    
    exp.pump1_max_vol = 10

# Peak finding function
This method is the same as the one above, but tries to maximize the sensor readout, instead of searching for a target readout.

In [ ]:
mybounds = (0.001, 0.999) # bounds of possible pump 1 power in the search of the target value
x_atol = (0.02) # absolute tolerance of pump power in search of target value
maxiter = 7 # maximum number of iterations allowed in search
            #these are also the default values

exp2.peak_finding_function(mybounds, x_atol, maxiter)

In [16]:
wn.Beep(700,2000)

# Ending the program
After map is created, or a particular experiment is concluded, the program has to be "closed". This simply corresponds to a closing of the open COM ports to the syringe pumps so that a new instance of the experiment class can be created. This is done with the command:
    
    exp.end()


In [ ]:
exp2.end()

# Post Experiment Data Loading

As previously mentioned, once an experiment is complete, but the "complex_experiment" class instance is still running, the program is able to load the data from the file structure it created. No additional implementation to navigate the files created is necessary. The program can also do this once the instance is gone i.e. the program can load data at any time, not only while the files are created. To do this, you need another instance of the experiment class, and you simply point it to the .pkl file of the experiment files you wish to load, with the load_dic_path argument. This argument should be the path to said "path_config" pickle file. 

In [ ]:
pump_dictionary # defined earlier
pump1_max_volume = 10 #in mL
pump2_max_volume = 10
pump3_max_volume = 10
pump4_max_volume = 10

overall_flow = 0.8  #in units set below
equilibration_time = 45  #in seconds
target_sensor_readout = 100  #explained later
path123 = os.path.dirname(os.path.abspath("2pump_control.py")) #any path could be used as input
units = "mL/min" #other option is μL/min
syringe_diameter = 15.68 #in mm
syringe_volume = 12  #in mL
_pumps = "OFF"  #should be "OFF" if pumps are not connected, but many features will not work

# Optional Arguments
communication_rate = 38400  # baudrate to be used
colour = "R"  # colour channel to be used by data loading and automations methods
load_dic_path = "C:\\Users\\Ignacy\\OneDrive - Imperial College London\\MSci\\Data\\Term 2\\Mixer V28\\Complex Pumpkin Experiment 2022-02-21\\path_config 15.57.pkl"

exp2_data_loading = complex_experiment(pump_dictionary, pump1_max_volume, pump2_max_volume, pump3_max_vol, pump4_max_vol, 
                          overall_flow, equilibration_time, target_sensor_readout, path123, units, 
                          syringe_diameter, syringe_volume, _pumps,
                          communication_rate, colour, load_dic_path)

After this, all the data loading features described earlier should work, and can be used for data analysis. 

## Automapping and Feature Searching

Now moving on to the most advanced feature of this program. The main assumption is that only the FRR of pump 3 & 4 is varied, and that the pump 1 & 2 FRR is kept constant. 

The program is a loop of logspace maps, where the program changes the bounds so that they are closer and closer to the interesting features found in the initial map. It does this by interpolating the points collected in the initial map with a cubic spline, and dividing that spine into 10 sections. Then, the variance of the y values in those sections is calculated. If the variance is < 9% of the maximum variance found in the 10 sections, that section is deemed uninteresting, and excluded from the next search.

However, not all sections can be rejected... the program analyzes the cubic spline first from left to right, and then from right to left. Therefore, any middle sections that are flat should not be excluded. 

The cubic spline is also limitted so that the beginning and end of the spline have a first derrivate of 0, i.e. the beginning and end of the interesting sections should resemble a flat line. 

This approach should be able to detect a number of different feature shapes, such as a sigmoid or a gaussian, and deem them as interesting. 

A different appraoch using block entrop was also tested, and it worked well on simulated data, but did not function of real world data.

### Using this function

This function, the pump3only_auto_mapping() function, takes almost the same input arguments as the complex_experiment class. This is because it actually calls on the complex_experiment class to perform the actual maps. But it takes a number of extra arguments as well:
    
    pump3_half_stock_conc -- concentration of the substance in the syringe in pump 3
                             taken so that graphs that are plotted have a accurate x-axis label
                             
    n_samples -- number of points to be taken for each map, recommended minimum of 6
    
    load_dic_path -- optional argument
                     path to path_config .pkl file, that can be used to start the program based on an existing map
                     this is very useful in the case that the program has to be interrupted because of e.g. the 
                     syringes running out of liquid
    
    initial_bounds -- optional argument
                      defualt values = (-3, -0.01)
                      The initial bounds set for the first map performed... 
                      ...if no path to a path_cofig pkl file is given.
                      Again, the bounds are logarithmic, i.e. to turn them into pump power, use 10^ (initial_bounds)

The way that running the function actually works is that, once you run it, it will ask you for the pump power of pump 1. After giving it the pump 1 power, you then have to:

If no previous map is used:

    You have to alt tab to a maximized Ximea CamTool and wait untill a map is collected. 
    Afterwords you have to return to the program.
    
After a map is collected, or loaded into the program
Two plots will be plotted, one with a logarithmic scale, and one with a concentration scale
On them, the proposed new map section will be highlighted.
The program will then wait for an empty input to continue.

This manual check was implemented as a development measure, to make monitoring the intial run of the function easier. However, it could easily be removed.

The program will continue reducing the range of the mapped space, until it deems all sections being mapped as information rich.                  

In [28]:
pump_dic = {'2': 'COM4', '4': 'COM5', '1': 'COM7', '3': 'COM8'}

pump1_max_vol = 3.8
pump2_max_vol = 12
pump3_max_vol = 8
pump4_max_vol = 12

_pumps = "ON"

number_of_pumps = 4
syringe_diameter = 15.89
syringe_volume = 12

overall_flow = 0.8
equilibriation_time = 45        ## 20 + 35
target_value = 100
units = "mL/min"
path123 = "C:\\Users\\Ignacy\\OneDrive - Imperial College London\\MSci\\Data\\Term 2\\Mixer V28"
communication_rate = 38400
colour = "R"
pump3_half_stock_conc = 0.15
n_samples = 8
load_dic_path = "C:\\Users\\Ignacy\OneDrive - Imperial College London\\MSci\\Data\\Term 2\\Mixer V28\\Complex Pumpkin Experiment 2022-03-23\\path_config 15.1.pkl"


In [ ]:
pump3only_auto_mapping(pump_dic, pump1_max_vol, pump2_max_vol, pump3_max_vol, pump4_max_vol, overall_flow, equilibriation_time, target_value, units, path123, communication_rate, colour, syringe_diameter, syringe_volume, _pumps, pump3_half_stock_conc, n_samples, load_dic_path)

Unfortunately, for this function to work, you need to be connected to the pumps. However, here is a screenshot of what it would look like: (seeker_finder was the intial name of the function)

![graph1](graph1.png)

Iteration number 2:

![graph2](graph2.png)

And so forth until the highlighted area would reach both edges
The data could then be loaded as described before, as each iteration would just be a new complex_experiment instance

Code developed by
    
    Ignacy Bartnik
    IgnacyABartnik@gmail.com
Supervised by

    Robert Strutt
    Nick Brooks
Created as part of
    
    Master Project
    MSci Chemistry with Molecular Physics
    Imperial College London